# MiaplPy

**MIAmi Phase Linking in PYthon**

An open source python package of InSAR processing with Non Linear phase inversion in full resolution. It reads a stack of coregistered SLCs and produces time series of surface deformation. This package depends on MintPy for time series corrections. It works with all data based on ISCE or GAMMA outputs format. The inversion is based on wrapped phase time series and it includes PTA (SqueeSAR), EMI (EVD+ML) and EVD (CAESAR) techniques. It also supports sequential inversion.

# General overview
The time series analysis workflow in miaplpyApp consists of 8 steps which will be explained one by one through an example dataset.

**Example data:**
The example data we use here is a stack of Sentinel-1 images in descending track 142 acquired over Guagua Pichincha volcano, Ecuador (Zenodo; ~318M in size). It spans from 2016.04.19 to 2017.12.28 with 46 acquisitions in total. We use the stack Sentinel processor (Fattahi et al., 2016) within ISCE (Rosen et al., 2012) for coregistering the SAR stack.

# 0. Initial setup of the notebook
The cell below performs the intial setup of the notebook and must be run every time the notebook (re)starts. It defines the processing location and check the example dataset. It is possible to partially complete the exercise, close the notebook, and come back and continue later from that point, but this initialization must be re-run.

In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from mintpy.objects import ifgramStack
from mintpy.utils import plot as pp, utils as ut
from mintpy import view, plot_network

# define work directory
work_dir = os.path.expanduser('~/data/test')
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)
print('Go to work directory: {}'.format(work_dir))

# download example dataset
url = 'https://zenodo.org/record/6539952/files/PichinchaSenDT142.zip'
zip_file = os.path.basename(url)
proj_name = zip_file.split('.')[0]
if not os.path.isfile(zip_file):
    !wget {url}
else:
    print('{} exists, skip re-download.'.format(zip_file))

# uncompress example dataset
if not os.path.isdir(proj_name):
    print('uncompressing the zip file ...')
    !unzip -q {zip_file}

# go to project directory
proj_dir = os.path.join(work_dir, proj_name)
os.chdir(proj_dir)
print('Go to project directory: {}'.format(proj_dir))

Go to work directory: /home/enr30065/data/test
PichinchaSenDT142.zip exists, skip re-download.
Go to project directory: /home/enr30065/data/test/PichinchaSenDT142


# 1. Setting up template file
The configuration parameters to miaplpyApp.py are controlled through three template files. At least one template is required to run miaplpyApp.py.

default template: miaplpyApp.cfg. It contains all MiaplPy configuration parameters, grouped by steps, with default auto values (which are defined in miaplpyApp_auto.cfg). This file is copied over to the current working directory and read every time miaplpyApp.py runs.

default template: smallbaselineApp.cfg. It contains all MintPy configuration parameters, grouped by steps, with default auto values (which are defined in smallbaselineApp_auto.cfg). This file is copied over to the current working directory and read every time smallbaselineApp.py runs.

custom template (optional but recommended): PichinchaSenDT142.template in the example dataset. It contains selective, manually modified configuration parameters. The custome template file name is arbitrary. Custom template has higher priority than the two default templates; if custom template is input, miaplpyApp.py will update the default miaplpyApp.cfg and smallbaselineApp.cfg files accordingly.

**Parallel processing**
If you want to run tasks in parallel, then you can set the number of workers/cores per node with miaplpy.compute.numCores option. The default is set to 4 for a personal computer with 4 CPUs.

# Custom template example
The path of all input files (miaplpy.load.*), subset area (miaplpy.subset.*) and the reference point (mintpy.reference.*) are always recommended.

# 2. miaplpyApp.py processing steps
The miaplpyApp.py workflow can be called with a single command-line call; by default it will run all the required processing steps with options pulled from the template files. However, in this notebook, we will use the "step" processing, this allows you to re-start the processing from a given step. If you work on HPC, you can create jobs for each step and run them one by one. More detailed usage can be found in help.

In [2]:
!miaplpyApp --help

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
usage: miaplpyApp [-h] [--dir WORKDIR] [-g] [-H] [-v] [--walltime WALL_TIME]
                  [--queue QUEUE] [--jobfiles] [--runfiles] [--tmp]
                  [--start STEP] [--end STEP] [--dostep STEP]
                  [customTemplateFile]

Routine Time Series Analysis for MiaplPy

positional arguments:
  customTemplateFile    Custom template with option settings.
                        ignored if the default miaplpyApp.cfg is input.

options:
  -h, --help            show this help message and exit
  --dir WORKDIR, --work-dir WORKDIR
                        Work directory, (default: None).
  -g                    Generate default template (if it does not exist) and exit.
  -H                    Print the default template file and exit.
  -v, --version         Print software version and exit
  --walltime WALL_TIME  walltime for submitting the script as a job
  --queue QUEUE         Queue name
  --j

## 2.1. Create processing steps run files
Running miaplpyApp.py with --runfiles will create all steps as separate runfiles and you can then run them one by one.



In [4]:
!miaplpyApp PichinchaSenDT142.txt --runfiles --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------

_________________________________________________      

  /##      /## /##                     / ## /#######
 | ###    /###|__/|                    | ##| ##__  ##
 | ####  /#### /##|/ ##### /##         | ##| ##  \ ## /##   /##
 | ## ##/## ##| ##| ##__  ### /####### | ##| #######/| ##  | ##
 | ##  ###| ##| ##| ##__  ###| ##__  ##| ##| ##      | ##  | ##
 | ##\  # | ##| ##| ##   \###| ##   \##| ##| ##      | ##  | ##
 | ## \/  | ##| ##|  ##### ##| #######/| ##| ##      |  #######
 |__/     |__/|__/| \____/|_/| ##____/ |__/|__/       \____  ##
                             | ##                     /##  | ## 
                             | ##                    |  ######/         
                             |__/                     \______/
 Miami Non-Linear Phase Linking software in Python   
          MiaplPy 0.2.1, 2023-08-22
__________________________

In [5]:
!ls -l miaplpy

total 460
-rw-r----- 1 enr30065 enr30065    244 Oct 23 20:00 conf.full
-rw-r----- 1 enr30065 enr30065  24639 Nov 14 14:09 custom_smallbaselineApp.cfg
drwxr-x--- 4 enr30065 enr30065   4096 Oct 23 20:03 inputs
drwxr-x--- 4 enr30065 enr30065   4096 Nov 13 14:45 inverted
-rw-r----- 1 enr30065 enr30065   4702 Nov 14 14:09 log
-rw-r----- 1 enr30065 enr30065 919904 Nov 13 14:45 maskPS.h5
-rw-r----- 1 enr30065 enr30065  69488 Nov 13 15:03 maskPS.png
-rw-r----- 1 enr30065 enr30065   7711 Oct 23 20:00 miaplpyApp.cfg
drwxr-x--- 6 enr30065 enr30065   4096 Nov 14 14:09 network_single_reference
-rw-r----- 1 enr30065 enr30065    274 Nov 14 14:09 run_files_list
-rw-r----- 1 enr30065 enr30065 919704 Nov 13 15:03 shadow_mask.h5
-rw-r----- 1 enr30065 enr30065  69330 Nov 13 15:03 shadow_mask.png


In [6]:
!ls -l miaplpy/network_single_reference/run_files

total 270
-rw-r----- 1 enr30065 enr30065   238 Nov 14 14:09 run_01_miaplpy_load_data
-rw-r----- 1 enr30065 enr30065   317 Nov 14 14:09 run_02_miaplpy_phase_linking
-rw-r----- 1 enr30065 enr30065   230 Nov 14 14:09 run_03_miaplpy_concatenate_patches
-rw-r----- 1 enr30065 enr30065 15597 Nov 14 14:09 run_04_miaplpy_generate_ifgram
-rw-r----- 1 enr30065 enr30065 27861 Nov 14 14:09 run_05_miaplpy_unwrap_ifgram
-rw-r----- 1 enr30065 enr30065   490 Nov 14 14:09 run_06_miaplpy_load_ifgram
-rw-r----- 1 enr30065 enr30065   218 Nov 14 14:09 run_07_mintpy_ifgram_correction
-rw-r----- 1 enr30065 enr30065   433 Nov 14 14:09 run_08_miaplpy_invert_network
-rw-r----- 1 enr30065 enr30065   187 Nov 14 14:09 run_09_mintpy_timeseries_correction


## 2.2 Load SLC and geometry stack (Run 1)
This step prepares the required metadata (using prep_isce.py) and load all data files and their metadata into HDF5 files in the ./inputs folder, including:

./inputs/slcStack.h5
./inputs/geometryRadar.h5  #for dataset in radar coordinates
./inputs/geometryGeo.h5    #for dataset in geo coordinates 

In [7]:
!miaplpyApp PichinchaSenDT142.txt --dostep load_data --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-14 14:12:32.641184--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['load_data']
Remaining steps: ['phase_linking', 'concatenate_patches', 'generate_ifgram', 'unwrap_ifgram', 'load_ifgram', 'ifgram_correction', 'invert_network', 'timeseries_correction']
Project name: PichinchaSenDT142
--------------------------------------------------
20241114:141232 * miaplpyApp.py PichinchaSenDT142.txt --dostep load_data --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/Pi

In [7]:
!ls -l miaplpy/inputs

total 322167
-rw-r--r--  1 enr30065 enr30065      1875 Apr 21  2022 PichinchaSenDT142.txt
drwxr-xr-x 53 enr30065 enr30065      4096 Feb 10  2022 baselines
-rw-r-----  1 enr30065 enr30065       413 Nov 13 14:34 date_list.txt
-rw-r-----  1 enr30065 enr30065  12228631 Nov 13 14:34 geometryRadar.h5
-rw-r-----  1 enr30065 enr30065      7711 Oct 23 20:00 miaplpyApp.cfg
drwxr-xr-x  3 enr30065 enr30065      4096 Feb 10  2022 reference
-rw-r-----  1 enr30065 enr30065 354562320 Nov 13 14:34 slcStack.h5
-rw-r-----  1 enr30065 enr30065     24639 Nov 13 14:34 smallbaselineApp.cfg


In [8]:
!info.py miaplpy/inputs/slcStack.h5

******************** Basic File Info ************************
file name: /home/enr30065/data/test/PichinchaSenDT142/miaplpy/inputs/slcStack.h5
file type: timeseries
coordinates : RADAR

******************** Date Stat Info *************************
Start Date: 20160419
End   Date: 20171228
Number of dates  : 46
STD of datetimes : 0.51 years

******************** HDF5 File Structure ********************
Attributes in / level:
  ALOOKS                    1
  ANTENNA_SIDE              -1
  AZIMUTH_PIXEL_SIZE        15.616311889140093
  BANDS                     1
  CENTER_INCIDENCE_ANGLE    46.153805
  CENTER_LINE_UTC           39141.0
  DATA_TYPE                 complex64
  DATE                      20170408
  EARTH_RADIUS              6337298.410627297
  FILE_LENGTH               530
  FILE_PATH                 /home/enr30065/data/test/PichinchaSenDT142/merged/SLC/20170408/20170408.slc.full
  FILE_TYPE                 timeseries
  HEADING                   -167.96890986886447
  HEIGHT   

In [9]:
!info.py ./miaplpy/inputs/geometryRadar.h5 --compact

******************** Basic File Info ************************
file name: /home/enr30065/data/test/PichinchaSenDT142/miaplpy/inputs/geometryRadar.h5
file type: geometry
coordinates : RADAR

******************** HDF5 File Structure ********************
Attributes in / level:
  ALOOKS                    1
  ANTENNA_SIDE              -1
  AZIMUTH_PIXEL_SIZE        15.616311889140093
  BANDS                     1
  CENTER_INCIDENCE_ANGLE    46.153805
  CENTER_LINE_UTC           39141.0
  DATA_TYPE                 complex64
  DATE                      20170408
  EARTH_RADIUS              6337298.410627297
  FILE_LENGTH               530
  FILE_PATH                 /home/enr30065/data/test/PichinchaSenDT142/merged/SLC/20170408/20170408.slc.full
  FILE_TYPE                 geometry
  HEADING                   -167.96890986886447
  HEIGHT                    698882.4836057822
  INTERLEAVE                BSQ
  LAT_REF1                  -0.14213333
  LAT_REF2                  -0.13301475
  LAT_REF

## 2.2 Phase inversion (Run_2)
This step runs the phase linking using wrapped phase series. following are the options used for this step. Phase linking methods supported by MiaplPy are EVD (Fornaro et al., 2015, EMI (Ansari et al., 2018) and PTA (Ferretti et al., 2011). All of them can be run in sequential mode (sequential_EVD, sequential_EMI and sequential_PTA) (Ansari et al., 2017). MiaplPy also supports SBW [Ansari et al., 2017] and you can choose a bandwidth (number of connections) by setting miaplpy.inversion.sbw_connNum option in template.

For sequential mode, you can set the number of images as ministack size by miaplpy.inversion.ministackSize. Range and azimuth window are the size of searching window to find statistical homogeneous pixels (SHPs). Shp test is the statistical test to find SHPs and three tests are supported by MiaplPy including KS (Stephens, 1970), AD (Scholz & Stephens, 1987) and ttest (Derrick et al., 2017).

This step supports parallel processing by dividing the subset area into patches of size n by n, for example the default size is 200 and the data will be divided into patches of size 200 by 200. You can set the number of parallel jobs (no more than available CPUs) using miaplpy.multiprocessing.numCores and the patch size by miaplpy.inversion.patchSize.

Depending on the number of parallel jobs, number of images, the processing method and the subset area (number of patches), the processing time varies. For one patch of 200*200 pixels and 46 images, processing time is 400 seconds (6 min) with a 2.8 GHz processor.

<img src="support/Phase_linking_algorithms.png" width="100%" />

In [10]:
!miaplpyApp PichinchaSenDT142.txt --dostep phase_linking --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-13 14:40:03.491510--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['phase_linking']
Remaining steps: ['concatenate_patches', 'generate_ifgram', 'unwrap_ifgram', 'load_ifgram', 'ifgram_correction', 'invert_network', 'timeseries_correction']
Project name: PichinchaSenDT142
--------------------------------------------------
20241113:144003 * miaplpyApp.py PichinchaSenDT142.txt --dostep phase_linking --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/PichinchaSe

## 2.3 Concatenate patches (Run_3)
After phase linking of all patches, this step would concatenate them

Either run:

miaplpyApp.py PichinchaSenDT142.txt --dostep concatenate_patches --dir ./miaplpy
or run the commands in run_03_miaplpy_concatenate_patches

In [11]:
!miaplpyApp PichinchaSenDT142.txt --dostep concatenate_patches --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-13 14:44:45.367885--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['concatenate_patches']
Remaining steps: ['generate_ifgram', 'unwrap_ifgram', 'load_ifgram', 'ifgram_correction', 'invert_network', 'timeseries_correction']
Project name: PichinchaSenDT142
--------------------------------------------------
20241113:144445 * miaplpyApp.py PichinchaSenDT142.txt --dostep concatenate_patches --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/PichinchaSenDT142.txt


In [12]:
!ls ./miaplpy/inverted

PATCHES				 tempCoh_average.aux.xml
amp_dipersion_index		 tempCoh_average.hdr
amp_dipersion_index.aux.xml	 tempCoh_full
amp_dipersion_index.hdr		 tempCoh_full.aux.xml
interferograms_single_reference  tempCoh_full.hdr
mask_unwrap			 top_eigenvalues
mask_unwrap.rsc			 top_eigenvalues.aux.xml
phase_series.h5			 top_eigenvalues.hdr
tempCoh_average


In [13]:
!ls ./miaplpy

conf.full		     log	     network_single_reference
custom_smallbaselineApp.cfg  maskPS.h5	     run_files_list
inputs			     maskPS.png      shadow_mask.h5
inverted		     miaplpyApp.cfg  shadow_mask.png


## 2.4 Select pairs to unwrap (generate_ifgram) (Run_4)
Since you are working in full resolution and for long temporal baselines, you may have low spatial coherence making unwrapping difficult. So in MiaplPy, you can select a network of minimum interferograms to unwrap and then convert phase to range changes.

For that, there are four options for which pairs to unwrap (following options in template). We use the single reference pairs for this example. The filter strength has been set to 0 by default but you may do additional filtering at this step.

<img src="support/2.png" width="100%" />

<img src="support/3.png" width="100%" />

In [21]:
!miaplpyApp PichinchaSenDT142.txt --dostep generate_ifgram --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-13 15:03:27.393354--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['generate_ifgram']
Remaining steps: ['unwrap_ifgram', 'load_ifgram', 'ifgram_correction', 'invert_network', 'timeseries_correction']
Project name: PichinchaSenDT142
--------------------------------------------------
20241113:150327 * miaplpyApp.py PichinchaSenDT142.txt --dostep generate_ifgram --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/PichinchaSenDT142.txt
update default template bas

In [22]:
!ls ./miaplpy/inverted/interferograms_single_reference

20170408_20160419  20170408_20161016  20170408_20170502  20170408_20170923
20170408_20160501  20170408_20161022  20170408_20170514  20170408_20171005
20170408_20160513  20170408_20161109  20170408_20170526  20170408_20171017
20170408_20160525  20170408_20161115  20170408_20170607  20170408_20171029
20170408_20160606  20170408_20161209  20170408_20170619  20170408_20171110
20170408_20160630  20170408_20170102  20170408_20170701  20170408_20171122
20170408_20160724  20170408_20170126  20170408_20170713  20170408_20171204
20170408_20160805  20170408_20170219  20170408_20170725  20170408_20171216
20170408_20160817  20170408_20170303  20170408_20170806  20170408_20171228
20170408_20160829  20170408_20170315  20170408_20170818
20170408_20160910  20170408_20170327  20170408_20170830
20170408_20160922  20170408_20170420  20170408_20170911


In [23]:
!cat ./miaplpy/conf.full

INITMETHOD	MCF
INITONLY    FALSE
INFILEFORMAT   COMPLEX_DATA
OUTFILEFORMAT	ALT_LINE_DATA
CORRFILEFORMAT	FLOAT_DATA
MINCONNCOMPFRAC 	0.01
SINGLETILEREOPTIMIZE   FALSE
STATCOSTMODE  DEFO
RMTMPTILE 	   TRUE
MAXNCOMPS       100
CONNCOMPTHRESH  300


## 2.5 Unwrap interferograms (Run_5)
For this step you may use your own unwrapping software. MiaplPy uses SNAPHU and you can set the following parameters for unwrapping. If you have a mask in HDF5 format like waterMask.h5, you can use it by miaplpy.unwrap.mask

In [24]:
!miaplpyApp PichinchaSenDT142.txt --dostep unwrap_ifgram --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-13 15:05:10.120025--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['unwrap_ifgram']
Remaining steps: ['load_ifgram', 'ifgram_correction', 'invert_network', 'timeseries_correction']
Project name: PichinchaSenDT142
--------------------------------------------------
20241113:150510 * miaplpyApp.py PichinchaSenDT142.txt --dostep unwrap_ifgram --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/PichinchaSenDT142.txt
update default template based on input custom te

## 2.6 Load interferograms in a stack (MintPy format) (run 6)
In this step you will load unwrapped interferograms to ./miaplpy/inputs/ifgramStack.h5, MintPy format. The available options are:

In [25]:
!miaplpyApp PichinchaSenDT142.txt --dostep load_ifgram --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-13 15:06:26.147047--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['load_ifgram']
Remaining steps: ['ifgram_correction', 'invert_network', 'timeseries_correction']
Project name: PichinchaSenDT142
--------------------------------------------------
20241113:150626 * miaplpyApp.py PichinchaSenDT142.txt --dostep load_ifgram --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/PichinchaSenDT142.txt
update default template based on input custom template
    mintpy.c

In [26]:
!ls -l ./miaplpy/network_single_reference/inputs

total 279053
-rw-r--r-- 1 enr30065 enr30065      1875 Apr 21  2022 PichinchaSenDT142.txt
-rw-r----- 1 enr30065 enr30065  12226695 Oct 30 19:48 geometryRadar.h5
-rw-r----- 1 enr30065 enr30065 334430197 Nov 13 15:07 ifgramStack.h5
-rw-r----- 1 enr30065 enr30065     24639 Oct 24 18:25 smallbaselineApp.cfg


## 2.7 Correct unwrapping error (using MintPy) (Run_7)
This step uses one of unwrapping error corrections of MintPy depending on the network of unwrapped interferograms. There are two methods of bridging and phase_closure or both of them to use. For a full description of the methods, refer to (Y. Zhang et al., 2019). If you unwrapped the short_baselines network, you can use both methods otherwise use bridging. You can set following parameters from smallbaselineApp.cfg:

In [27]:
!miaplpyApp PichinchaSenDT142.txt --dostep ifgram_correction --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-13 15:08:03.454715--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['ifgram_correction']
Remaining steps: ['invert_network', 'timeseries_correction']
Project name: PichinchaSenDT142
--------------------------------------------------
20241113:150803 * miaplpyApp.py PichinchaSenDT142.txt --dostep ifgram_correction --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/PichinchaSenDT142.txt
update default template based on input custom template
    mintpy.compute.cl

In [29]:
!info.py ./miaplpy/network_single_reference/inputs/ifgramStack.h5 --compact

******************** Basic File Info ************************
file name: /home/enr30065/data/test/PichinchaSenDT142/miaplpy/network_single_reference/inputs/ifgramStack.h5
file type: ifgramStack
coordinates : RADAR

******************** HDF5 File Structure ********************
Attributes in / level:
  ALOOKS                    1
  ANTENNA_SIDE              -1
  AZIMUTH_PIXEL_SIZE        15.616311889140093
  BANDS                     2
  CENTER_INCIDENCE_ANGLE    46.153805
  CENTER_LINE_UTC           39141.0
  DATA_TYPE                 float32
  DATE12                    170408-160419
  EARTH_RADIUS              6337298.410627297
  FILE_LENGTH               530
  FILE_PATH                 /home/enr30065/data/test/PichinchaSenDT142/miaplpy/inverted/interferograms_single_reference/20170408_20160419/filt_fine.unw
  FILE_TYPE                 ifgramStack
  HEADING                   -167.96890986886447
  HEIGHT                    698882.4836057822
  INTERLEAVE                BIL
  LAT_REF1    

## 2.8 Invert network and convert phase to range change time series
At this step you convert the ifgramStack.h5 to timeseries.h5 using minimum-norm deformation phase. Water mask and threshold for temporal coherence can be set with these options:

In [30]:
!miaplpyApp PichinchaSenDT142.txt --dostep invert_network --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-13 15:10:24.461643--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['invert_network']
Remaining steps: ['timeseries_correction']
Project name: PichinchaSenDT142
--------------------------------------------------
20241113:151024 * miaplpyApp.py PichinchaSenDT142.txt --dostep invert_network --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/PichinchaSenDT142.txt
update default template based on input custom template
    mintpy.compute.cluster: auto --> local
  

## 2.9 Time series Corrections (optional)
This step would apply different timeseries corrections from MintPy. Since the outputs are in the same format as MintPy, you can switch on any of the correction methods. In fact you can use smallbaselineApp.py instead and run the steps after correct_LOD.

Run smallbaselineApp.py -h for more help on the correction steps.

For this example we would only correct for topographic phase residuals and all others are set to no in the template file.

In [31]:
!miaplpyApp PichinchaSenDT142.txt --dostep timeseries_correction --dir ./miaplpy

Using default MintPy Path: /home/enr30065/miaplypyenv/lib/python3.10/site-packages
--------------------------------------------------
MiaplPy version v0.2.1, date 2023-08-22
--RUN-at-2024-11-13 15:13:06.432540--
Current directory: /home/enr30065/data/test/PichinchaSenDT142
Run routine processing with arg_parser.py on steps: ['timeseries_correction']
Remaining steps: []
Project name: PichinchaSenDT142
--------------------------------------------------
20241113:151306 * miaplpyApp.py PichinchaSenDT142.txt --dostep timeseries_correction --dir ./miaplpy
Project name: PichinchaSenDT142
Go to work directory: /home/enr30065/data/test/PichinchaSenDT142/miaplpy
copy default template file /home/enr30065/miaplypyenv/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /home/enr30065/data/test/PichinchaSenDT142/PichinchaSenDT142.txt
update default template based on input custom template
    mintpy.compute.cluster: auto --> local
    mintpy.

# 3. Correct for geolocation
Working in full resolution, required preceise geolocation specially in urban areas. This is a separate post processing step you may want to perform.

After topographic residual correction step of MintPy, There is demErr.h5 as a by-product. You can use it as bellow to correct the geo coordinates stored in ./miaplpy/network_single_reference/inputs/geometryRadar.h5.

correct_geolocation.py -g ./miaplpy/network_single_reference/inputs/geometryRadar.h5 -d ./miaplpy/network_single_reference/demErr.h5
Remember: This correction is in radar coordinate and you need to run it before geocode step in MintPy, or run the geocoding again:

smallbaselineApp.py PichinchaSenDT142.txt --dir ./miaplpy --start geocode

In [32]:
!pwd

/home/enr30065/data/test/PichinchaSenDT142


In [33]:
!ls

PichinchaSenDT142.txt  baselines  merged  miaplpy  reference
